<a href="https://colab.research.google.com/github/tharangag/Pyspark/blob/main/DS_Sales_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This demonstrates how to use PySpark to read external data and to create relational data frames
# Dataset Path and Tables
# /content/GS Sales Data 2022.xlsx
# Sheet Names
# 1> GS Sales Data
# 2> Daily Target

In [1]:
# Install and Configure PySpark on Colab
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 58.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9315182d6e4b33e6d6ac04c85bdede53382283bcd048cf062dfe13ca5594596c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
# Load Library
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import to_date, col
# Build Spark Session
spark = SparkSession.builder.appName('DS Internship Task1').getOrCreate()

In [3]:
# Read Data from Dataset i.e. Excel file : Sheet "GS Sales Data"
GS_Sales_Pdf = pd.read_excel('/content/GS Sales Data 2022.xlsx',sheet_name='GS Sales Data', index_col=0,
                             na_values='#N/A')
# Display top 5 record loaded into Pandas DF
GS_Sales_Pdf.head()


,Order ID,Order Date,Order_Week,Ship Date,Ship_Week,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,...,Regional Manager,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Returned
Row ID,,,,,,,,,,,,,,,,,,,,,
1,US-2019-103800,2019-01-03,2019-1,2019-01-07,2019-2,Standard Class,DP-13000,Darren Powers,Consumer,United States,...,Roxanne Rodriguez,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512,NaN
2,US-2019-112326,2019-01-04,2019-1,2019-01-08,2019-2,Standard Class,PO-19195,Phillina Ober,Home Office,United States,...,Roxanne Rodriguez,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870,NaN
3,US-2019-112326,2019-01-04,2019-1,2019-01-08,2019-2,Standard Class,PO-19195,Phillina Ober,Home Office,United States,...,Roxanne Rodriguez,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717,NaN
4,US-2019-112326,2019-01-04,2019-1,2019-01-08,2019-2,Standard Class,PO-19195,Phillina Ober,Home Office,United States,...,Roxanne Rodriguez,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748,NaN
5,US-2019-141817,2019-01-05,2019-1,2019-01-12,2019-2,Standard Class,MB-18085,Mick Brown,Consumer,United States,...,Chuck Magee,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840,NaN


In [4]:
# Cast field data types on pandas DF to String due to some fields contain "#N/A" values to avoid data type conversion errors in spark
GS_Sales_Pdf[['State/Province', 'Country/Region','Regional Manager','Customer Name','Returned','Category']] = GS_Sales_Pdf[['State/Province', 'Country/Region','Regional Manager','Customer Name','Returned','Category']].astype(str)


In [5]:
# Create Spark Dataframe
GS_Sales = spark.createDataFrame(GS_Sales_Pdf)

# View first 20 rows in spark dataframe 
GS_Sales.show()

+--------------+-------------------+----------+-------------------+---------+--------------+-----------+----------------+-----------+--------------+------------+--------------+-----------+-------+-----------------+---------------+---------------+------------+--------------------+------------------+--------+--------+-------------------+--------+
|      Order ID|         Order Date|Order_Week|          Ship Date|Ship_Week|     Ship Mode|Customer ID|   Customer Name|    Segment|Country/Region|        City|State/Province|Postal Code| Region| Regional Manager|     Product ID|       Category|Sub-Category|        Product Name|             Sales|Quantity|Discount|             Profit|Returned|
+--------------+-------------------+----------+-------------------+---------+--------------+-----------+----------------+-----------+--------------+------------+--------------+-----------+-------+-----------------+---------------+---------------+------------+--------------------+------------------+-------

In [6]:
# View spark Data Frame Schema
GS_Sales.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Order Date: timestamp (nullable = true)
 |-- Order_Week: string (nullable = true)
 |-- Ship Date: timestamp (nullable = true)
 |-- Ship_Week: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State/Province: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Regional Manager: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Returned: string (nullable = t

In [7]:
# Create New Spark Dataframe and Transform Column Headings and Data Types

GS_Sales_finalDF = (GS_Sales.withColumn("order_id", GS_Sales["Order ID"]).drop("Order ID")
              .withColumn("order_date", to_date(col("Order Date"), "mm/dd/yyyy")).drop("Order Date")
              .withColumn("order_week_no", GS_Sales["Order_Week"]).drop("Order_Week")
              .withColumn("ship_date", to_date(col("Ship Date"), "mm/dd/yyyy")).drop("Ship Date")
              .withColumn("ship_week_no", GS_Sales["Ship_Week"]).drop("Ship_Week")
              .withColumn("ship_mode", GS_Sales["Ship Mode"]).drop("Ship Mode")
              .withColumn("customer_id", GS_Sales["Customer ID"]).drop("Customer ID")
              .withColumn("customer_name", GS_Sales["Customer Name"]).drop("Customer Name")
              .withColumn("customer_segment", GS_Sales["Segment"]).drop("Segment")
              .withColumn("country_region", GS_Sales["Country/Region"]).drop("Country/Region")
              .withColumn("customer_city", GS_Sales["City"]).drop("City")
              .withColumn("state_province", GS_Sales["State/Province"]).drop("State/Province")
              .withColumn("postal_code", GS_Sales["Postal Code"]).drop("Postal Code")
              .withColumn("customer_region", GS_Sales["Region"]).drop("Region")
              .withColumn("regional_manager", GS_Sales["Regional Manager"]).drop("Regional Manager")
              .withColumn("product_id", GS_Sales["Product ID"]).drop("Product ID")
              .withColumn("product_category", GS_Sales["Category"]).drop("Category")
              .withColumn("sub_category", GS_Sales["Sub-Category"]).drop("Sub-Category")
              .withColumn("product_name", GS_Sales["Product Name"]).drop("Product Name")
              .withColumn("sales_revenue", GS_Sales["Sales"].cast('float')).drop("Sales")
              .withColumn("sales_quantity", GS_Sales["Quantity"].cast("float")).drop("Quantity")
              .withColumn("discount_amount", GS_Sales["Discount"].cast('float')).drop("Discount")
              .withColumn("sales_profit", GS_Sales["Profit"].cast('float')).drop("Profit")
              .drop("Returned")
              .distinct()
            ) 

GS_Sales_finalDF.show(10)

+--------------+----------+-------------+----------+------------+--------------+-----------+----------------+----------------+--------------+-------------+--------------+-----------+---------------+-----------------+---------------+----------------+------------+--------------------+-------------+--------------+---------------+------------+
|      order_id|order_date|order_week_no| ship_date|ship_week_no|     ship_mode|customer_id|   customer_name|customer_segment|country_region|customer_city|state_province|postal_code|customer_region| regional_manager|     product_id|product_category|sub_category|        product_name|sales_revenue|sales_quantity|discount_amount|sales_profit|
+--------------+----------+-------------+----------+------------+--------------+-----------+----------------+----------------+--------------+-------------+--------------+-----------+---------------+-----------------+---------------+----------------+------------+--------------------+-------------+--------------+----

In [8]:
GS_Sales_finalDF.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_week_no: string (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- ship_week_no: string (nullable = true)
 |-- ship_mode: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_segment: string (nullable = true)
 |-- country_region: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- state_province: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- customer_region: string (nullable = true)
 |-- regional_manager: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales_revenue: float (nullable = true)
 |-- sales_quantity: float (nullable = true)
 |-- discount_amount: float (nullable = true)
 |-- sales_profit: flo

In [9]:
# Create spark SQL Select query to retrieve customer data as a DataFrame from a temporary table view
GS_Sales_finalDF.createOrReplaceTempView("sales")

DimCustomer_sqlDF = spark.sql("SELECT customer_id,	customer_name FROM sales")
DimCustomer_sqlDF.show()

+-----------+-------------------+
|customer_id|      customer_name|
+-----------+-------------------+
|   MS-17830|      Melanie Seite|
|   KN-16450|        Kean Nguyen|
|   SP-20650|   Stephanie Phelps|
|   CT-11995|       Carol Triggs|
|   EH-14185|         Evan Henry|
|   FM-14215|      Filia McAdams|
|   PV-18985|      Paul Van Hugh|
|   RS-19420|    Ricardo Sperren|
|   SM-20950|     Suzanne McNair|
|   AP-10915|     Arthur Prichep|
|   CM-12715|     Craig Molinari|
|   ST-20530|           Shui Tom|
|   ON-18715|      Odella Nelson|
|   CB-12025|  Cassandra Brandow|
|   VD-21670|  Valerie Dominguez|
|   AB-10105|      Adrian Barton|
|   PJ-18835|      Patrick Jones|
|   AG-10300|Aleksandra Gannaway|
|   CA-11965|        Carol Adams|
|   KM-16720|       Kunst Miller|
+-----------+-------------------+
only showing top 20 rows



In [10]:
# Create spark SQL query to create a Dim Table as a DataFrame from Transformed Data Frame

DimCustomer_DF=spark.sql("SELECT DISTINCT customer_id as CustomerIDSk,	customer_name as CustomerName,  postal_code as AddressKey FROM sales")

# Print Schema of DIM Table/DataFrame
DimCustomer_DF.printSchema()

# Display top 20 Records
DimCustomer_DF.show()

root
 |-- CustomerIDSk: string (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- AddressKey: string (nullable = true)

+------------+----------------+----------+
|CustomerIDSk|    CustomerName|AddressKey|
+------------+----------------+----------+
|    JD-15895|Jonathan Doherty|     19140|
|    RB-19465|    Rick Bensley|     74133|
|    RR-19525|       Rick Reed|     01852|
|    ON-18715|   Odella Nelson|     85281|
|    SB-20290|    Sean Braxton|     77036|
|    KM-16660|    Khloe Miller|     95123|
|    KL-16555|   Kelly Lampkin|     80906|
|    GA-14725|   Guy Armstrong|     19120|
|    VW-21775| Victoria Wilson|     44256|
|    CS-12250| Chris Selesnick|     55125|
|    KB-16315|      Karl Braun|     11520|
|    SR-20740|   Steven Roelle|     10024|
|    GZ-14470|   Gary Zandusky|     87124|
|    AG-10900|   Arthur Gainer|     19134|
|    TS-21160|   Theresa Swint|     17403|
|    AY-10555|      Andy Yotov|     90049|
|    SP-20650|Stephanie Phelps|     28403|
|   

In [11]:
# Create spark SQL query to create a Fact Table as a DataFrame from Transformed Data Frame

FactOrders_DF=spark.sql("SELECT DISTINCT order_id as OrderID,	order_date as OrderDate,  ship_date as ShipDate, ship_week_no as ShipWeek, ship_mode as ShipMode, customer_id as CustomerIDKey, product_id as ProductIDKey, sales_revenue as Sales, sales_quantity as Quantity, discount_amount as Discount, sales_profit as Profit FROM sales")

# Print Schema of DIM Table/DataFrame
FactOrders_DF.printSchema()

# Display top 20 Records
FactOrders_DF.show()

root
 |-- OrderID: string (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- ShipDate: date (nullable = true)
 |-- ShipWeek: string (nullable = true)
 |-- ShipMode: string (nullable = true)
 |-- CustomerIDKey: string (nullable = true)
 |-- ProductIDKey: string (nullable = true)
 |-- Sales: float (nullable = true)
 |-- Quantity: float (nullable = true)
 |-- Discount: float (nullable = true)
 |-- Profit: float (nullable = true)

+--------------+----------+----------+--------+--------------+-------------+---------------+-------+--------+--------+--------+
|       OrderID| OrderDate|  ShipDate|ShipWeek|      ShipMode|CustomerIDKey|   ProductIDKey|  Sales|Quantity|Discount|  Profit|
+--------------+----------+----------+--------+--------------+-------------+---------------+-------+--------+--------+--------+
|US-2019-162775|2019-01-13|2019-01-15|  2019-3|  Second Class|     CS-12250|OFF-ST-10000025| 572.58|     6.0|     0.0| 34.3548|
|US-2019-163223|2019-03-21|2019-03-25| 2019-13